In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [0]:
import pandas as pd
import os
import numpy as np
import cv2

In [0]:
def read_data(fin):
    """ 이미지 파일을 읽어 들여 이미지 데이터, 국가를 리턴"""
    data = pd.read_csv(fin)
    target_li=[]
    data_li=[]
    for i in range(len(data)):
        image_id = int(data.iloc[i][0])
        target_nation = data.iloc[i][3]
        image_path = '/content/drive/My Drive/valid_picture/'+ target_nation +'/{}.png'.format(image_id)
       
        if (os.path.isfile(image_path)):
            image_data = cv2.imread(image_path)
            data_li.append(image_data)
            target_li.append(target_nation)
          #  print(image_id)

    return (np.array(data_li), np.array(target_li))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
def create_train_test_data(image_data, label_li):
    #데이터 수, 세로 픽셀, 가로 픽셀 
    n_samples, image_h, image_w, _channel = image_data.shape
    print(image_h)
    print(image_w)
    X = image_data
 
    n_features = X.shape[1]
    y = label_li
    #클래스 갯수 = 분류할 국가 수
    n_classes = 5
    
    print("total dataset size:")
    print("n_samples: %d" % n_samples)
    print("n_features: %d" % n_features)
    print("n_classes: %d" % n_classes)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    return(X_train, X_test, y_train, y_test)

In [0]:
def load_VGG(X_train, X_test, y_train, y_test, epochs):
  img_width, img_height = 48, 48
  batch_size = 16
  #vVGG모델 불러옴.
  vgg_model = applications.VGG16(weights='imagenet',  include_top = False, input_shape = (img_width, img_height, 3))

  
  #레이어 4개 동결
  for layer in vgg_model.layers[:4]:
      layer.trainable = False
#레이어 3개 동결
#  for layer in vgg_model.layers[:3]:
#      layer.trainable = False

#레이어 2개 동결
#  for layer in vgg_model.layers[:2]:
#      layer.trainable = False
      
  #예측 레이어 연결해줌
  x = vgg_model.output
  x = Flatten()(x)
  x = Dense(1024, activation="relu")(x)
  predictions = Dense(5, activation="softmax")(x)
  
  model_final = Model(input = vgg_model.input, output = predictions)
  model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
  vgg_model.summary()
  model_final.fit(X_train, y_train, batch_size = batch_size , epochs=epochs)
  
  test_loss, test_acc = model_final.evaluate(X_test, y_test)

  print('테스트 정확도:', test_acc)

In [1]:
def load_VGG_4block(X_train, X_test, y_train, y_test, epochs):
  img_width, img_height = 48, 48
  batch_size = 16
  #vVGG모델 불러옴.
  vgg_model = applications.VGG16(weights='imagenet',  include_top = False, input_shape = (img_width, img_height, 3))
  
  #마지막층 블럭5 - conv, pool 빼버림
  vgg_model.layers.pop()
  vgg_model.layers.pop()
  vgg_model.layers.pop()
  vgg_model.layers.pop()
  
  #레이어 2개 동결
  for layer in vgg_model.layers[:2]:
      layer.trainable = False  
   
  #예측 레이어 연결해줌
  x = vgg_model.output
  x = Flatten()(x)
  x = Dense(1024, activation="relu")(x)
  predictions = Dense(5, activation="softmax")(x)
  
  model_final = Model(input = vgg_model.input, output = predictions)
  model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
  vgg_model.summary()
  model_final.fit(X_train, y_train, batch_size = batch_size , epochs=epochs)
  
  test_loss, test_acc = model_final.evaluate(X_test, y_test)

  print('테스트 정확도:', test_acc)

In [162]:
#데이터 읽어오고,
image_data, label = read_data('5country.csv')
#pandas의 omne-hot-encoding
label = pd.get_dummies(label)
#X_train = X_train/255.0
#y_train = y_train/255.0
# train-test 나눠줌
X_train, X_test, y_train, y_test = create_train_test_data(image_data, label)

48
48
total dataset size:
n_samples: 4805
n_features: 48
n_classes: 5


In [157]:
#위에서 정의한 모델 불러오기 #epoch 15에 5층 레이어
load_VGG(X_train, X_test, y_train, y_test, 15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [161]:
#위에서 정의한 모델 불러오기 #5층 epoch 15. x값 /255
load_VGG(X_train, X_test, y_train, y_test, 15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [163]:
#위에서 정의한 모델 불러오기 #5층
load_VGG(X_train, X_test, y_train, y_test, 50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [164]:
#위에서 정의한 모델 불러오기 #5층
load_VGG(X_train, X_test, y_train, y_test, 15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [170]:
#위에서 정의한 모델 불러오기 #5층 레이어 2개만 동결
load_VGG(X_train, X_test, y_train, y_test, 20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [171]:
#위에서 정의한 모델 불러오기 #5층 레이어 4개 동결
load_VGG(X_train, X_test, y_train, y_test, 10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [166]:
load_VGG_4block(X_train, X_test, y_train, y_test, 20)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________